In [2]:
import igraph as ig
import pandas as pd
import numpy as np
from pathlib import Path
import networkx as nx
import scipy.sparse as sp
from src.config import INTERIM_DATA_DIR
import matplotlib.pyplot as plt

## 1. 数据准备与联合网络建模
#### 读取图结构

In [3]:
# 电-气图结构位置
g_path = INTERIM_DATA_DIR / "Texas7k_Gas/merged_graph.pkl"
# 读取图结构
g = ig.Graph.Read_Pickle(g_path)

# 获取所有节点和边的信息
nodes = g.vs  # 节点列表
edges = g.es  # 边列表

In [4]:
# 节点距离矩阵，形状为为 N节点×N节点
distances_upper = sp.load_npz(INTERIM_DATA_DIR / "Texas7k_Gas/distances.npz").toarray()
distances = distances_upper + distances_upper.T
# 邻接矩阵, 形状为 N节点×N节点
adj_sparse = sp.load_npz(INTERIM_DATA_DIR / "Texas7k_Gas/adj_sparse.npz")
print(f"邻接矩阵形状: {adj_sparse.shape}")
# 一阶边界矩阵，节点到边的关联矩阵，形状为 N节点×N边
B1 = sp.load_npz(INTERIM_DATA_DIR / "Texas7k_Gas/B1.npz")
print(f"一阶边界矩阵形状: {B1.shape}")
# 一阶拉普拉斯矩阵，形状为 N边×N边
L1_tilde = sp.load_npz(INTERIM_DATA_DIR / "Texas7k_Gas/L1_tilde.npz")
print(f"一阶拉普拉斯矩阵形状: {L1_tilde.shape}")

邻接矩阵形状: (9168, 9168)
一阶边界矩阵形状: (9168, 11667)
一阶拉普拉斯矩阵形状: (11667, 11667)


In [5]:
edge_feature_names = np.load(INTERIM_DATA_DIR / "Texas7k_Gas/edge_feature_names.npy", allow_pickle=True)
edge_features = np.load(INTERIM_DATA_DIR / "Texas7k_Gas/edge_features.npy", allow_pickle=True)
node_feature_names = np.load(INTERIM_DATA_DIR / "Texas7k_Gas/node_feature_names.npy", allow_pickle=True)
node_features = np.load(INTERIM_DATA_DIR / "Texas7k_Gas/node_features.npy", allow_pickle=True)
# 组成DataFrame
edge_features_df = pd.DataFrame(edge_features, columns=edge_feature_names)
node_features_df = pd.DataFrame(node_features, columns=node_feature_names)
# 打印数据集的基本信息
print(f"边特征数据集形状: {edge_features_df.shape}")
print(f"节点特征数据集形状: {node_features_df.shape}")
# 打印数据集的前几行
print("边特征数据集前5行:")
print(edge_features_df.head())
print("节点特征数据集前5行:")
print(node_features_df.head())
# 打印数据集的描述信息
print("边特征数据集描述信息:")
print(edge_features_df.describe())
print("节点特征数据集描述信息:")
print(node_features_df.describe())

边特征数据集形状: (11667, 9)
节点特征数据集形状: (9168, 25)
边特征数据集前5行:
         r        x        b  capacity    q  length  diameter  \
0  0.00144  0.04775  0.00000     245.7  0.0     0.0       0.0   
1  0.01608  0.08810  0.05448     227.9  0.0     0.0       0.0   
2  0.00148  0.00896  0.01119     307.2  0.0     0.0       0.0   
3  0.01094  0.05109  0.07783     257.2  0.0     0.0       0.0   
4  0.00616  0.04105  0.01329     271.4  0.0     0.0       0.0   

   friction_factor    k  
0              0.0  0.0  
1              0.0  0.0  
2              0.0  0.0  
3              0.0  0.0  
4              0.0  0.0  
节点特征数据集前5行:
  latitude longitude   base_kv        pd        qd has_generator  \
0 -0.28121   0.30762  0.501981 -0.556817  -0.40192         False   
1 -0.28121   0.30762 -1.348365 -0.568059 -0.412471         False   
2 -0.28121   0.30762 -1.348365 -0.568059 -0.412471         False   
3 -0.28121   0.30762 -1.539061 -0.568059 -0.412471         False   
4 -0.28121   0.30762  0.501981  3.504815  4.686